# Single Responsibility Principle (SRP)

> “A class should have only one reason to change.” – Robert C. Martin

## 1. Concept
* A **single responsibility** means the class encapsulates one and only one **axis of change**.
* Fewer responsibilities → higher cohesion and lower coupling.

## 2. Problematic design without SRP

In [1]:
// BAD: One class doing too many things
class OrderService {
  private db: Record<string, any>[] = [];

  // 1. Business rule
  calculateTotal(order: { items: { price: number; quantity: number }[] }): number {
    return order.items.reduce((sum, i) => sum + i.price * i.quantity, 0);
  }

  // 2. Persistence
  save(order: any) {
    this.db.push(order);
  }

  // 3. External communication
  sendConfirmationEmail(order: any) {
    console.log(`Email sent to ${order.customerEmail}`);
  }

  process(order: any) {
    order.total = this.calculateTotal(order);
    this.save(order);
    this.sendConfirmationEmail(order);
  }
}

const service = new OrderService();
service.process({
  customerEmail: "alice@example.com",
  items: [
    { price: 10, quantity: 3 },
    { price: 30, quantity: 1 }
  ]
});

Email sent to alice@example.com


### Why is this design fragile?
* **Tight coupling**: business rules, persistence, and messaging are combined.
* **Difficult to test**: you can’t unit‑test calculation without touching the console or fake DB.
* **Hard to change**: a change in storage technology or email service forces editing this class.

## 3. Applying SRP – refactor

In [2]:
// GOOD: each class handles one responsibility
class OrderCalculator {
  total(order: { items: { price: number; quantity: number }[] }): number {
    return order.items.reduce((sum, i) => sum + i.price * i.quantity, 0);
  }
}

class OrderRepository {
  private db: Record<string, any>[] = [];
  save(order: any) {
    this.db.push(order);
  }
}

class EmailService {
  send(to: string, content: string) {
    console.log(`Sending email to ${to}: ${content}`);
  }
}

class OrderProcessor {
  constructor(
    private calculator: OrderCalculator,
    private repo: OrderRepository,
    private emailer: EmailService
  ) {}

  process(order: any) {
    order.total = this.calculator.total(order);
    this.repo.save(order);
    this.emailer.send(order.customerEmail, `Your total is $${order.total}`);
  }
}

const processor = new OrderProcessor(
  new OrderCalculator(),
  new OrderRepository(),
  new EmailService()
);

processor.process({
  customerEmail: "alice@example.com",
  items: [
    { price: 10, quantity: 3 },
    { price: 30, quantity: 1 }
  ]
});

Sending email to alice@example.com: Your total is $60


### Benefits achieved
* **Isolated change**: storage or email mechanism can evolve independently.
* **Testability**: calculator can be tested with pure data; repository/email can be mocked.
* **Reusability**: the same `EmailService` can be reused by other features.

## 4. Trade‑offs and limitations
* **More classes & files** – mental overhead for small scripts.
* **Requires composition** – you need a dependency‑injection pattern or manual wiring.
* **Can be over‑applied** – splitting too early may lead to needless indirection.

## 5. Checklist for spotting multiple responsibilities
1. Does the class change for more than one stakeholder (e.g., DBA & UX team)?
2. Does it read/write from more than one data source?
3. Does it format UI and perform calculations?
If **yes**—split!

## 6. References
* Robert C. Martin, *Clean Architecture* (2017)
* https://principles.dev
* https://en.wikipedia.org/wiki/Single-responsibility_principle